In [117]:
import pandas as pd
import numpy as np
import os
import json
import requests
import gc
from tqdm import tqdm
from datetime import datetime, timedelta
pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


In [2]:
lct = "/Users/nam-yeong/Downloads/balso_tmp"
folders = [file for file in os.listdir(lct) if file.endswith('.csv')]
folders.sort()
for i in folders:
    print(os.path.join(lct,i))
    


/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-08_00_00_00+00_00-2024-04-09_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_33b25716-bac1-47e6-b040-be837fbb5ce6.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-09_00_00_00+00_00-2024-04-10_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_495ee3b4-e43f-422d-84d1-e0ec813345dd.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-10_00_00_00+00_00-2024-04-11_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_05214fdf-838f-4c99-a0ed-6b6d2b5fdb0d.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-11_00_00_00+00_00-2024-04-12_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_c662fe3f-b9e2-48dc-be8c-c4d35252edec.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-12_00_00_00+00_00-2024-04-13_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_79232362-6c0b-42d1-b3f8-6c5d8c927a38.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-13_00_00_00+00_00-2024-04-14_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_2c7ec2ba-edd6-4

Clicl에 상품정보 담기지 않은 개발 이슈. Impression > CLick 퍼널 이어진 데이터를 선별해서 상품 정보 Stitching

In [18]:
tst = folders[-1]
# chunksize = 5000
# nrows = 1000
df = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{tst}")
df.query("`Event Category`.str.contains('AD')")
conv_users = list(set(df.query("`Event Category`.str.contains('Click')")['Airbridge Device ID'].tolist()))
df = df.query("`Airbridge Device ID` in @conv_users")
df[(df['Event Category'].str.contains('Impression') )& (df['Event Datetime']>='2024-04-08T23:59:50+09:00')]

df.drop_duplicates(subset=['Airbridge Device ID','Event Datetime','Event Category','Product List'],inplace=True)


In [21]:
len(df[df['Event Category'].str.contains('Click')])

57458

In [19]:
df.sort_values(by=['Airbridge Device ID', 'Event Datetime','Event Category'],ascending=[True,True,False], inplace=True)
df['prev_prop'] = df.groupby('Airbridge Device ID')['Product Name'].shift(1)
# df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]
df = df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]

df

,Event Category,Event Action,Event Label,Event Value,Event Date,Event Datetime,Semantic Event Properties,Custom Event Properties,Total Revenue,Total Revenue (Original),Currency,Currency (Original),Product List,Product ID,Product Name,Product Price,Product Quantity,Product Currency,Product Position,Product Brand ID,Product Brand Name,Product Category ID,Product Category Name,Airbridge Device ID,Platform,User ID,prev_prop
403757,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T19:29:18+09:00,"{""products"":[{""name"":""프리미엄 인도산 강황정 강황환 HACCP 인...",{},NaN,NaN,NaN,NaN,"[{""name"":""프리미엄 인도산 강황정 강황환 HACCP 인증 500mg 300정...",NaN,"프리미엄 인도산 강황정 강황환 HACCP 인증 500mg 300정, 150g, 2개",42300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0002CD6F-4106-464E-89BD-C726A1A2879D,iOS,caa65f1e-2436-4c04-a8da-78bd3968aab7,"프리미엄 인도산 강황정 강황환 HACCP 인증 500mg 300정, 150g, 2개"
65800,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T23:26:13+09:00,"{""products"":[{""name"":""테디베리 국내산 딸기청 1+1, 550g, ...",{},NaN,NaN,NaN,NaN,"[{""name"":""테디베리 국내산 딸기청 1+1, 550g, 2병"",""price"":...",NaN,"테디베리 국내산 딸기청 1+1, 550g, 2병",17820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00034858-1022-4103-AE9D-AEA98E20A495,iOS,38855c94-50e5-4142-aa98-8593171f59ea,"테디베리 국내산 딸기청 1+1, 550g, 2병"
688634,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T13:04:52+09:00,"{""products"":[{""name"":""본필 6인치 작업안전화 BSS805 발이편한...",{},NaN,NaN,NaN,NaN,"[{""name"":""본필 6인치 작업안전화 BSS805 발이편한 작업화, 1개"",""p...",NaN,"본필 6인치 작업안전화 BSS805 발이편한 작업화, 1개",39800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0003a68a-7269-47c2-ba1a-3c47d6044103,Android,cc593bfe-2ff5-414c-95c6-7aca3b043240,"본필 6인치 작업안전화 BSS805 발이편한 작업화, 1개"
814660,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T04:51:23+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00069c4c-3eb0-4efc-a949-96ffda9cb8ef,Android,4bd4896f-f23e-44dd-bf54-ec084ef1f7be,두부김치만두 소포장 묶음 느끼하지 않고 담백한 강원도식 만두 먹어도 먹어도 질리지 ...
814654,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T04:51:27+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00069c4c-3eb0-4efc-a949-96ffda9cb8ef,Android,4bd4896f-f23e-44dd-bf54-ec084ef1f7be,곰곰 소중한 우리쌀밥
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736015,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T11:20:57+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff1c7de-1495-488d-89e2-ea3295d11ebd,Android,18d10bca-7631-4341-9156-9a14c2058aec,[HOTAX] 세련된 클래식 접이식 미니벨로 자전거 출퇴근 경량 미니 폴딩 바구니 ...
262376,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T21:24:51+09:00,"{""products"":[{""name"":""프리미엄 로즈힙 퓨레 엘라스틴 600mg 6...",{},NaN,NaN,NaN,NaN,"[{""name"":""프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘...",NaN,"프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘라스틴 60정 2병",31330.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff5e906-06ea-44ed-ae01-c4c3e6c4a821,Android,66482903-6256-4092-9d20-1e220804f289,"프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘라스틴 60정 2병"
497648,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T17:52:33+09:00,"{""products"":[{""name"":""여성 봄 정장 자켓"",""price"":6300...",{},NaN,NaN,NaN,NaN,"[{""name"":""여성 봄 정장 자켓"",""price"":63000}]",NaN,여성 봄 정장 자켓,63000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff8f4f4-8256-4df3-a923-fef6b12a090d,Android,047d6ed4-a627-4d73-bc59-1467313bfde3,여성 봄 정장 자켓
243776,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T21:37:36+09:00,"{""products"":[{""name"":""메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹...",{},NaN,NaN,NaN,NaN,"[{""name"":""메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정,...",NaN,"메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정, 다크브라운, 3개",29100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fffaeb83-177a-4747-bb1d-61b9e9377010,Android,ff8d11c2-4265-4f9c-9388-80790643f799,"메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정, 다크브라운, 3개"


In [22]:
len(df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]) 

57458

In [20]:
df.to_parquet(f'./{folders[i]}_conv.parquet')

자정에 데이터 끊기는 이슈. 타겟 date, d+1 데이터 로드해서 처리 후, d+1 데이터는 burn \
마지막 데이터의 경우, Click에 상품정보 포함되어 있음. 따라서 burn 이후, 정상 데이터 merge

In [17]:

for i in tqdm(range(len(folders))):
    if i < len(folders):
        
        tg = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{folders[i]}")
        tmp = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{folders[i+1]}")
        tmp = tmp[(tmp['Event Category'].str.contains('Clicks')) & (tmp['Event Datetime'] < '2024-04-09T00:30:00+09:00')]
        df = pd.concat([tg,tmp],axis=0)
        
        del tmp, tg
        gc.collect()
    else:
        df = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{folders[i]}")
        
    conv_users = list(set(df.query("`Event Category`.str.contains('Click')")['Airbridge Device ID'].tolist()))
    df = df.query("`Airbridge Device ID` in @conv_users")
    df.drop_duplicates(subset=['Airbridge Device ID','Event Datetime','Event Category','Product List'],inplace=True)
    
    print("전 클릭 전환 : ",len(df[df['Event Category'].str.contains('Click')]))
    
    df.sort_values(by=['Airbridge Device ID', 'Event Datetime','Event Category'],ascending=[True,True,False], inplace=True)
    df['prev_prop'] = df.groupby('Airbridge Device ID')['Product Name'].shift(1)
    df = df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]
    
    print("후 클릭 전환 : ",len(df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]))
    print("유실율 : ",len(df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]) / len(df[df['Event Category'].str.contains('Click')]))
    
    df.to_parquet(f'./{folders[i]}_conv.parquet')
        

  0%|          | 0/12 [00:00<?, ?it/s]

전 클릭 전환 :  124142
후 클릭 전환 :  121547
유실율 :  1.0


  8%|▊         | 1/12 [02:38<29:01, 158.30s/it]

전 클릭 전환 :  106669
후 클릭 전환 :  105124
유실율 :  1.0


 17%|█▋        | 2/12 [06:13<31:56, 191.61s/it]

...

전 클릭 전환 :  121020
후 클릭 전환 :  118663
유실율 :  1.0


 83%|████████▎ | 10/12 [38:25<07:34, 227.41s/it]

전 클릭 전환 :  70993
후 클릭 전환 :  70218
유실율 :  1.0


 92%|█████████▏| 11/12 [45:52<04:10, 250.25s/it]


IndexError: list index out of range

In [26]:
os.getcwd()

'/Users/nam-yeong/git/bayes-ABtest/cp_test'

In [53]:
lct = "."
folders = [file for file in os.listdir(lct) if file.endswith('.parquet')]
folders.sort()
for i in folders:

    print(os.path.join(lct,i))

./20-21_data.parquet
./balso_2024-04-08_00_00_00+00_00-2024-04-09_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_33b25716-bac1-47e6-b040-be837fbb5ce6.parquet
./balso_2024-04-09_00_00_00+00_00-2024-04-10_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_495ee3b4-e43f-422d-84d1-e0ec813345dd.parquet
./balso_2024-04-10_00_00_00+00_00-2024-04-11_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_05214fdf-838f-4c99-a0ed-6b6d2b5fdb0d.parquet
./balso_2024-04-11_00_00_00+00_00-2024-04-12_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_c662fe3f-b9e2-48dc-be8c-c4d35252edec.parquet
./balso_2024-04-12_00_00_00+00_00-2024-04-13_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_79232362-6c0b-42d1-b3f8-6c5d8c927a38.parquet
./balso_2024-04-13_00_00_00+00_00-2024-04-14_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_2c7ec2ba-edd6-4766-a4de-f568be1f058c.parquet
./balso_2024-04-14_00_00_00+00_00-2024-04-15_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_11f57b85-f208-460c-86b8-bd68c16eee52.parquet

In [54]:
lct = "."
folders = [file for file in os.listdir(lct) if file.endswith('.parquet')]
folders.sort()
for i in folders:

    print(os.path.join(lct,i))

df = pd.DataFrame()

for i in folders:
    tmp = pd.read_parquet(os.path.join(lct,i))
    print(len(tmp))
    df= pd.concat([df,tmp],axis=0)
    
df


./20-21_data.parquet
./balso_2024-04-08_00_00_00+00_00-2024-04-09_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_33b25716-bac1-47e6-b040-be837fbb5ce6.parquet
./balso_2024-04-09_00_00_00+00_00-2024-04-10_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_495ee3b4-e43f-422d-84d1-e0ec813345dd.parquet
./balso_2024-04-10_00_00_00+00_00-2024-04-11_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_05214fdf-838f-4c99-a0ed-6b6d2b5fdb0d.parquet
./balso_2024-04-11_00_00_00+00_00-2024-04-12_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_c662fe3f-b9e2-48dc-be8c-c4d35252edec.parquet
./balso_2024-04-12_00_00_00+00_00-2024-04-13_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_79232362-6c0b-42d1-b3f8-6c5d8c927a38.parquet
./balso_2024-04-13_00_00_00+00_00-2024-04-14_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_2c7ec2ba-edd6-4766-a4de-f568be1f058c.parquet
./balso_2024-04-14_00_00_00+00_00-2024-04-15_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_11f57b85-f208-460c-86b8-bd68c16eee52.parquet

,Event Category,Event Action,Event Label,Event Value,Event Date,Event Datetime,Semantic Event Properties,Custom Event Properties,Total Revenue,Total Revenue (Original),Currency,Currency (Original),Product List,Product ID,Product Name,Product Price,Product Quantity,Product Currency,Product Position,Product Brand ID,Product Brand Name,Product Category ID,Product Category Name,Airbridge Device ID,Platform,User ID,prev_prop
2,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-20,2024-04-20T23:59:59+09:00,"{""products"":[{""name"":""GUTFLEX 것플렉스 두부오트스낵 두부과자...",{},NaN,NaN,NaN,NaN,"[{""name"":""GUTFLEX 것플렉스 두부오트스낵 두부과자 사무실 단백질 식이섬...",NaN,GUTFLEX 것플렉스 두부오트스낵 두부과자 사무실 단백질 식이섬유 과자 건강 간식...,8800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53F0FCC2-0462-47E0-9C5F-96C8EBD1F25A,iOS,26835191-af83-4470-a231-36f92f12bb72,NaN
12,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-20,2024-04-20T23:59:59+09:00,"{""products"":[{""name"":""팬톤 ANC 액티브 노이즈캔슬링 블루투스5....",{},NaN,NaN,NaN,NaN,"[{""name"":""팬톤 ANC 액티브 노이즈캔슬링 블루투스5.3 TWS 이어폰 PT...",NaN,"팬톤 ANC 액티브 노이즈캔슬링 블루투스5.3 TWS 이어폰 PTB-06, Calm...",19920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d54b1f64-a4ac-4195-a800-145ef690bff3,Android,4994f035-0963-4569-a30e-a5934c7401b0,NaN
20,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-20,2024-04-20T23:59:59+09:00,"{""products"":[{""name"":""소웨이브 인체공학 발편한 푹신한 구름 깔창""...",{},NaN,NaN,NaN,NaN,"[{""name"":""소웨이브 인체공학 발편한 푹신한 구름 깔창"",""price"":190...",NaN,소웨이브 인체공학 발편한 푹신한 구름 깔창,19000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dcbe500c-7a9f-417f-851e-8d4cd1eceb07,Android,88acbe75-2aa0-418e-9212-a9dfcfddd953,NaN
58,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-20,2024-04-20T23:59:58+09:00,"{""products"":[{""name"":""국산 원홀 싱크대 수전 씽크대 수도꼭지 70...",{},NaN,NaN,NaN,NaN,"[{""name"":""국산 원홀 싱크대 수전 씽크대 수도꼭지 7020, 1세트"",""pr...",NaN,"국산 원홀 싱크대 수전 씽크대 수도꼭지 7020, 1세트",47700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BB2A21BA-1D99-4460-9063-921F5893083E,iOS,969d2a1f-a7e4-4fed-b4fe-c8929c0f4379,NaN
62,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-20,2024-04-20T23:59:57+09:00,"{""products"":[{""name"":""합체 선글라스"",""price"":12900}]}",{},NaN,NaN,NaN,NaN,"[{""name"":""합체 선글라스"",""price"":12900}]",NaN,합체 선글라스,12900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3524af48-d059-4714-bb23-d3909e94a17e,Android,5f11d496-eb00-4e3f-a7f9-3e08bf21f506,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736015,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T11:20:57+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff1c7de-1495-488d-89e2-ea3295d11ebd,Android,18d10bca-7631-4341-9156-9a14c2058aec,[HOTAX] 세련된 클래식 접이식 미니벨로 자전거 출퇴근 경량 미니 폴딩 바구니 ...
262376,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T21:24:51+09:00,"{""products"":[{""name"":""프리미엄 로즈힙 퓨레 엘라스틴 600mg 6...",{},NaN,NaN,NaN,NaN,"[{""name"":""프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘...",NaN,"프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘라스틴 60정 2병",31330.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff5e906-06ea-44ed-ae01-c4c3e6c4a821,Android,66482903-6256-4092-9d20-1e220804f289,"프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘라스틴 60정 2병"
497648,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T17:52:33+09:00,"{""products"":[{""name"":""여성 봄 정장 자켓"",""price"":6300...",{},NaN,NaN,NaN,NaN,"[{""name"":""여성 봄 정장 자켓"",""price"":63000}]",NaN,여성 봄 정장 자켓,63000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff8f4f4-8256-4df3-a923-fef6b12a090d,Android,047d6ed4-a627-4d73-bc59-1467313bfde3,여성 봄 정장 자켓
243776,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T21:37:36+09:00,"{""products"":[{""name"":""메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹...",{},NaN,NaN,NaN,NaN,"[{""name"":""메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정,...",NaN,"메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정, 다크브라운, 3개",29100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fffaeb83-177a-4747-bb1d-61b9e9377010,Android,ff8d11c2-4265-4f9c-9388-80790643f799,"메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정, 다크브라운, 3개"


In [55]:
df.drop_duplicates(df,inplace=True)

19일까지는 클릭에 데이터가 없으니 위의 방법으로 처리함. 20일부터는 클릭에 데이터 들어가있으므로 추가

In [103]:
# lct = "."
# folders = [file for file in os.listdir(lct) if file.endswith('.csv')]
# folders.sort()
# for i in folders:

#     print(os.path.join(lct,i))

# df_append = pd.DataFrame()

# for i in folders:
#     tmp = pd.read_csv(os.path.join(lct,i))
#     print(len(tmp))
#     tmp = tmp.query("`Event Category`.str.contains('Click')")
#     df_append= pd.concat([df_append,tmp],axis=0)
    
# df_append

# df_append['prev_prop'] = df_append['Product Name']
# df_append.to_parquet('./20-21_data.parquet')

# df_append = pd.read_parquet('./20-21_data.parquet')
# df = pd.concat([df,df_append],axis=0)
# del df_append
# gc.collect()

In [210]:
df.columns

Index(['Event Category', 'Event Action', 'Event Label', 'Event Value',
       'Event Date', 'Event Datetime', 'Semantic Event Properties',
       'Custom Event Properties', 'Total Revenue', 'Total Revenue (Original)',
       'Currency', 'Currency (Original)', 'Product List', 'Product ID',
       'Product Name', 'Product Price', 'Product Quantity', 'Product Currency',
       'Product Position', 'Product Brand ID', 'Product Brand Name',
       'Product Category ID', 'Product Category Name', 'Airbridge Device ID',
       'Platform', 'User ID', 'prev_prop'],
      dtype='object')

In [186]:
df['Event Date'].unique()


array(['2024-04-08', '2024-04-09', '2024-04-10', '2024-04-11',
       '2024-04-12', '2024-04-13', '2024-04-14', '2024-04-15',
       '2024-04-16', '2024-04-17', '2024-04-18', '2024-04-19',
       '2024-04-20', '2024-04-21'], dtype=object)

쿠팡 처리

In [5]:
access_key = pd.read_csv('../../cp_data.csv').iloc[0,1]
secret_key = pd.read_csv('../../cp_data.csv').iloc[1,1]

In [147]:
import requests
import hmac
import hashlib
from time import gmtime, strftime

REQUEST_METHOD = "GET"
DOMAIN = "https://api-gateway.coupang.com"
URL = "/v2/providers/affiliate_open_api/apis/openapi/v1/reports/orders"

# Replace with your own ACCESS_KEY, SECRET_KEY, startDate, endDate
ACCESS_KEY = access_key
SECRET_KEY = secret_key
startDate = "20240408"
endDate = "20240422"
# Assuming subId is not required, if needed add it to the query parameters

def generateHmac(method, url, secretKey, accessKey, queryString=""):
    # Correct formatting for date-time if needed
    datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
    message = datetimeGMT + method + url + queryString

    signature = hmac.new(bytes(secretKey, 'utf-8'), 
                         message.encode('utf-8'), 
                         hashlib.sha256).hexdigest()

    return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetimeGMT, signature)


def fetch_data(page):
    queryString = "startDate={}&endDate={}&page={}".format(startDate, endDate, page)
    fullURL = "{}{}?{}".format(DOMAIN, URL, queryString)
    authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
    response = requests.request(method=REQUEST_METHOD, url=fullURL,
                                headers={
                                    "Authorization": authorization,
                                    "Content-Type": "application/json"
                                })
    return response.json()

def main():
    page = 1
    results = []

    while True:
        response = fetch_data(page)
        if 'data' in response:
            results.extend(response['data'])
            if len(response['data']) < 1000:
                break
            page += 1
        else:
            # Handle errors or missing 'data' key
            print(f"Error retrieving data: {response.get('message', 'No error message available')}")
            break

    print(f"Total items fetched: {len(results)}")
    return results


# Execute the main function to handle pagination
all_data = main()


Total items fetched: 37536


In [148]:
cp = pd.DataFrame(all_data)
cp['date'] = pd.to_datetime(cp['date'])
cp.columns
len(cp)

cp.rename(columns={'date':'Event Date', 'subParam':'User ID', 'productName':'prev_prop'},inplace=True)
df['Event Date'] = pd.to_datetime(df['Event Date'])


In [165]:
tst = cp['User ID'].unique().tolist()
tst = df['User ID'].unique().tolist()
cp[~cp['User ID'].isin(tst)]['User ID'].nunique() / cp['User ID'].nunique()


0.18447902710501907

In [194]:
tst = list(set(cp.subParam.tolist()))

df.query('`User ID`in @tst')['User ID'].nunique()

11493

이상?함?

In [145]:
tg_date = df['Event Date'].unique()

for i in tg_date:
    j = i + timedelta(days=1)
    df_burn = df[(df['Event Date']) == i]
    cp_burn = cp[(cp['Event Date']==i)]
    cp_burn_d = cp[(cp['Event Date']==j)]

    tst = list(set(cp_burn['User ID'].tolist()))
    tst_d = list(set(cp_burn_d['User ID'].tolist()))
    df_burn.query('`User ID`in @tst | `User ID`in @tst_d')['User ID'].nunique()

    print(i, len(tst) + len(tst_d), df_burn.query('`User ID`in @tst | `User ID`in @tst_d')['User ID'].nunique(),\
        
        )

2024-04-08 00:00:00 0 0
2024-04-09 00:00:00 0 0
2024-04-10 00:00:00 0 0
2024-04-11 00:00:00 1381 679
2024-04-12 00:00:00 2734 1564
2024-04-13 00:00:00 2662 1424
2024-04-14 00:00:00 2662 1320
2024-04-15 00:00:00 2846 1542
2024-04-16 00:00:00 3040 1697
2024-04-17 00:00:00 2718 1658
2024-04-18 00:00:00 1985 949
2024-04-19 00:00:00 1506 671
2024-04-20 00:00:00 1327 504
2024-04-21 00:00:00 635 312


In [146]:
cp

,Event Date,trackingCode,subId,User ID,addtag,ctag,orderId,productId,prev_prop,quantity,gmv,commissionRate,commission
10842,2024-04-12,AF3155932,homecoin,bdaceb8d-3ea3-45ca-afde-ca09661083a7,460,7974963103,8074944516100772,7670376838,양반 단호박죽,6,10060.0,5.25,529.0
10843,2024-04-12,AF3155932,homecoin,0c089cc6-ff4e-4637-8c68-1549d696bb3f,460,7041059088,8814944016749336,7692530333,[식약처기능인정] 뱃살타파 2주완성 마시는 레몬밤 오비엑스 워터믹스 클래식,1,23500.0,5.25,1234.0
10844,2024-04-12,AF3155932,homecoin,614de0bf-119e-4df8-85b1-017420830eed,460,6489890342,8364941016020644,7292466704,엘라스틴 데일리 리페어링 퍼퓸 샴푸+린스 1100ml 대용량,1,16430.0,5.25,863.0
10845,2024-04-12,AF3155932,homecoin,e2b9c525-2b2d-4154-ae4a-772d08a5b414,460,6990601217,8004943116446031,6320364957,워너프 탱크 베어 링 휴대폰 케이스,1,8910.0,5.25,468.0
10846,2024-04-12,AF3155932,homecoin,b077a3e4-d241-4a2d-9e41-1c120bbae424,460,6957293598,8604948416503210,140401459,미샤 롱웨어 젤 펜슬 라이너 0.14g,1,3940.0,5.25,207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35784,2024-04-21,AF3155932,homecoin,223d1d94-a139-45e2-923e-eb7253ba00b2,460,7684709310,8834949618374765,6312011497,부산 13번 중매인 생물 조림용 고등어 중 (3미),1,8390.0,5.25,441.0
35785,2024-04-21,AF3155932,homecoin,e54f90df-c83c-4b54-803e-14b0634b61ee,400,6597825235,8354944012301839,1658983488,라 아부엘라 니에베스 수제 감자칩,3,15240.0,5.25,801.0
35786,2024-04-21,AF3155932,homecoin,3049e76f-86fd-4850-bbca-ab903436022f,400,6170483695,8694941118973998,58558710,오븐락 내열유리 렌지 오븐 전용쿡 1.5L,1,9370.0,5.25,492.0
35787,2024-04-21,AF3155932,homecoin,2adab5cb-6b8f-4094-b056-42f0b6feb339,460,7883618811,8074944418455110,7329414983,산리오 소프트 유아동 레인부츠 어린이 장화,1,15780.0,5.25,829.0


로데이터로 테스트

In [20]:
lct = "/Users/nam-yeong/Downloads/balso_tmp"
folders = [file for file in os.listdir(lct) if file.endswith('.csv')]
folders.sort()
for i in folders:
    print(os.path.join(lct,i))
    


df_tst = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{folders[-1]}")


/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-08_00_00_00+00_00-2024-04-09_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_33b25716-bac1-47e6-b040-be837fbb5ce6.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-09_00_00_00+00_00-2024-04-10_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_495ee3b4-e43f-422d-84d1-e0ec813345dd.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-10_00_00_00+00_00-2024-04-11_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_05214fdf-838f-4c99-a0ed-6b6d2b5fdb0d.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-11_00_00_00+00_00-2024-04-12_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_c662fe3f-b9e2-48dc-be8c-c4d35252edec.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-12_00_00_00+00_00-2024-04-13_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_79232362-6c0b-42d1-b3f8-6c5d8c927a38.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-13_00_00_00+00_00-2024-04-14_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_2c7ec2ba-edd6-4

In [22]:
i = "2024-04-21"
cp_burn = cp[(cp['date']==i)]

tst = list(set(cp_burn.subParam.tolist()))
df_tst.query('`User ID`in @tst')['User ID'].nunique()

print(i, len(tst), df_burn.query('`User ID`in @tst')['User ID'].nunique())

2024-04-21 635 174


In [26]:
df['Event Date'].unique()

array(['2024-04-20', '2024-04-21', '2024-04-08', '2024-04-09',
       '2024-04-10', '2024-04-11', '2024-04-12', '2024-04-13',
       '2024-04-14', '2024-04-15', '2024-04-16', '2024-04-17',
       '2024-04-18', '2024-04-19'], dtype=object)

In [25]:
df.query('`User ID`.isna()')['Airbridge Device ID'].nunique() / df['Airbridge Device ID'].nunique()

0.6931717021091975

In [49]:
ddd = "2024-04-13"
df.query('`Event Date`==@ddd & `User ID`.isna()')['Airbridge Device ID'].nunique() / df.query('`Event Date`==@ddd')['Airbridge Device ID'].nunique()

df.query('`Event Date`==@ddd')['Airbridge Device ID'].nunique()

99695

10,11일 이슈 제외, 나머지기간의 unique user는 85% 맞음... 날짜이슈? 뭐지

In [203]:
tst = list(set(cp.query('date >= "2024-04-12"') .subParam.tolist()))

len(tst), df.query('`Event Date` >="2024-04-12" & `User ID`in @tst')['User ID'].nunique()

(10093, 8547)

In [125]:
df[df['Event Date']=='2024-04-19']['User ID'].nunique()

49865

#### 유실 데이터가 이상함. 클릭으로 보자

In [132]:
lct = "/Users/nam-yeong/Downloads/balso_tmp"
folders = [file for file in os.listdir(lct) if file.endswith('.csv')]
folders.sort()
for i in folders:
    print(os.path.join(lct,i))
    

tst = folders[-1]
# chunksize = 5000
# nrows = 1000
df_tst = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{tst}")
df_tst.query("`Event Category`.str.contains('AD')")
conv_users = list(set(df_tst.query("`Event Category`.str.contains('Click')")['Airbridge Device ID'].tolist()))
df_tst = df_tst.query("`Airbridge Device ID` in @conv_users")

/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-08_00_00_00+00_00-2024-04-09_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_33b25716-bac1-47e6-b040-be837fbb5ce6.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-09_00_00_00+00_00-2024-04-10_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_495ee3b4-e43f-422d-84d1-e0ec813345dd.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-10_00_00_00+00_00-2024-04-11_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_05214fdf-838f-4c99-a0ed-6b6d2b5fdb0d.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-11_00_00_00+00_00-2024-04-12_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_c662fe3f-b9e2-48dc-be8c-c4d35252edec.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-12_00_00_00+00_00-2024-04-13_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_79232362-6c0b-42d1-b3f8-6c5d8c927a38.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-13_00_00_00+00_00-2024-04-14_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_2c7ec2ba-edd6-4

In [135]:
df_tst = df_tst[df_tst['Event Category']=='Ad Click (App)']
df_tst

,Event Category,Event Action,Event Label,Event Value,Event Date,Event Datetime,Semantic Event Properties,Custom Event Properties,Total Revenue,Total Revenue (Original),Currency,Currency (Original),Product List,Product ID,Product Name,Product Price,Product Quantity,Product Currency,Product Position,Product Brand ID,Product Brand Name,Product Category ID,Product Category Name,Airbridge Device ID,Platform,User ID
0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T23:59:59+09:00,"{""products"":[{""name"":""고성능 어싱테스트 어싱테스기 어싱측정기 어싱...",{},NaN,NaN,NaN,NaN,"[{""name"":""고성능 어싱테스트 어싱테스기 어싱측정기 어싱 접지테스터기 확인기,...",NaN,"고성능 어싱테스트 어싱테스기 어싱측정기 어싱 접지테스터기 확인기, 1개",18350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3EBB6FED-801B-47E6-B8E3-7B28FD1BB1A2,iOS,6aa423dd-8831-4f54-a66a-55220a2cdc02
11,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T23:59:59+09:00,"{""products"":[{""name"":""라벨영 쇼킹헤어한방울앰플 4개"",""price...",{},NaN,NaN,NaN,NaN,"[{""name"":""라벨영 쇼킹헤어한방울앰플 4개"",""price"":29070}]",NaN,라벨영 쇼킹헤어한방울앰플 4개,29070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E5CBAF53-FF6E-4D2A-903C-02F93E930BA4,iOS,6a8edbbc-35fb-4138-8dca-8a83d7453b84
12,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T23:59:59+09:00,"{""products"":[{""name"":""[블랙라벨] 당도최상 블랙라벨 네이블 오렌지...",{},NaN,NaN,NaN,NaN,"[{""name"":""[블랙라벨] 당도최상 블랙라벨 네이블 오렌지, 1개, 3kg"",""...",NaN,"[블랙라벨] 당도최상 블랙라벨 네이블 오렌지, 1개, 3kg",23600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ebb0b70f-aa0b-4c4e-bc8f-d552fff2691f,Android,80335cb3-6fb9-4e2e-9c73-0d13b9c617ec
14,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T23:59:59+09:00,"{""products"":[{""name"":""오뚜기 수향미밥"",""price"":15700}]}",{},NaN,NaN,NaN,NaN,"[{""name"":""오뚜기 수향미밥"",""price"":15700}]",NaN,오뚜기 수향미밥,15700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C2A8AC04-D809-41AC-92E9-0706265E5751,iOS,ef6919c8-b09d-4cda-a3d9-e7c55acfbf31
17,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T23:59:59+09:00,"{""products"":[{""name"":""미스터장어 국산 자포니카 민물장어 1kg 손...",{},NaN,NaN,NaN,NaN,"[{""name"":""미스터장어 국산 자포니카 민물장어 1kg 손질 후 750g내외, ...",NaN,"미스터장어 국산 자포니카 민물장어 1kg 손질 후 750g내외, 1kg (손질후 7...",85800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6204b662-c245-4e61-8c65-23067eb830ec,Android,86a79644-283e-4f15-88ab-7c210bae7597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824311,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T00:00:01+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FAC3E5D4-E259-4860-89FC-DFB08348C328,iOS,c725b54d-6e08-4f76-89b3-79c95e54528a
824319,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T00:00:01+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04f41ea4-bbb4-4e9b-9c9a-116d9bf303f2,Android,000c5481-5502-4f13-8175-4c6dbfa739e9
824348,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T00:00:00+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f831f951-f199-4c37-94ad-bec7fe0dfbff,Android,5eb6e552-9ce0-4d5e-b205-5152de5f6a85
824349,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T00:00:00+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5e264395-4276-49ec-9fcd-ba3f15e593da,Android,10743edd-b487-430d-8a7a-4ff435c59f94


In [144]:
sp = cp.query('date == "2024-04-19"')['subParam'].unique()
df_tst.query('`User ID` in @sp')['User ID'].nunique() / len(sp)

/var/folders/7z/944bgjcs659fsp58nc954qgm0000gn/T/ipykernel_28023/3975437945.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  sp = cp.query('date == "2024-04-19"')['subParam'].unique()


0.4520884520884521

In [157]:
sp_ = df_tst[df_tst['Product Name'].notna()]['User ID'].unique().tolist()
cp.query('subParam in @sp_ & date=="2024-04-19"')['subParam'].nunique(), len(sp)

/var/folders/7z/944bgjcs659fsp58nc954qgm0000gn/T/ipykernel_28023/756292425.py:2: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  cp.query('subParam in @sp_ & date=="2024-04-19"')['subParam'].nunique(), len(sp)


(287, 814)

In [145]:
df['Event Date'].unique()

array(['2024-04-08', '2024-04-09', '2024-04-10', '2024-04-11',
       '2024-04-12', '2024-04-13', '2024-04-14', '2024-04-15',
       '2024-04-16', '2024-04-17', '2024-04-18', '2024-04-19'],
      dtype=object)

클릭으로도 50% 이상이 날라간다. 전체로 보면 77%는 있다. 왜? 쿠팡과 Datetime이 안맞을 수 있음. \
또는 클릭의 유실이 너무 많이 발생할 수 있음. 하지만 5% 이상의 클릭 유실은 발생할 수 없음.

In [166]:
df[df['Event Date']=='2024-04-19']

,Event Category,Event Action,Event Label,Event Value,Event Date,Event Datetime,Semantic Event Properties,Custom Event Properties,Total Revenue,Total Revenue (Original),Currency,Currency (Original),Product List,Product ID,Product Name,Product Price,Product Quantity,Product Currency,Product Position,Product Brand ID,Product Brand Name,Product Category ID,Product Category Name,Airbridge Device ID,Platform,User ID,prev_prop
403757,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T19:29:18+09:00,"{""products"":[{""name"":""프리미엄 인도산 강황정 강황환 HACCP 인...",{},NaN,NaN,NaN,NaN,"[{""name"":""프리미엄 인도산 강황정 강황환 HACCP 인증 500mg 300정...",NaN,"프리미엄 인도산 강황정 강황환 HACCP 인증 500mg 300정, 150g, 2개",42300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0002CD6F-4106-464E-89BD-C726A1A2879D,iOS,caa65f1e-2436-4c04-a8da-78bd3968aab7,"프리미엄 인도산 강황정 강황환 HACCP 인증 500mg 300정, 150g, 2개"
65800,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T23:26:13+09:00,"{""products"":[{""name"":""테디베리 국내산 딸기청 1+1, 550g, ...",{},NaN,NaN,NaN,NaN,"[{""name"":""테디베리 국내산 딸기청 1+1, 550g, 2병"",""price"":...",NaN,"테디베리 국내산 딸기청 1+1, 550g, 2병",17820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00034858-1022-4103-AE9D-AEA98E20A495,iOS,38855c94-50e5-4142-aa98-8593171f59ea,"테디베리 국내산 딸기청 1+1, 550g, 2병"
688634,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T13:04:52+09:00,"{""products"":[{""name"":""본필 6인치 작업안전화 BSS805 발이편한...",{},NaN,NaN,NaN,NaN,"[{""name"":""본필 6인치 작업안전화 BSS805 발이편한 작업화, 1개"",""p...",NaN,"본필 6인치 작업안전화 BSS805 발이편한 작업화, 1개",39800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0003a68a-7269-47c2-ba1a-3c47d6044103,Android,cc593bfe-2ff5-414c-95c6-7aca3b043240,"본필 6인치 작업안전화 BSS805 발이편한 작업화, 1개"
814660,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T04:51:23+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00069c4c-3eb0-4efc-a949-96ffda9cb8ef,Android,4bd4896f-f23e-44dd-bf54-ec084ef1f7be,두부김치만두 소포장 묶음 느끼하지 않고 담백한 강원도식 만두 먹어도 먹어도 질리지 ...
814654,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T04:51:27+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00069c4c-3eb0-4efc-a949-96ffda9cb8ef,Android,4bd4896f-f23e-44dd-bf54-ec084ef1f7be,곰곰 소중한 우리쌀밥
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736015,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T11:20:57+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff1c7de-1495-488d-89e2-ea3295d11ebd,Android,18d10bca-7631-4341-9156-9a14c2058aec,[HOTAX] 세련된 클래식 접이식 미니벨로 자전거 출퇴근 경량 미니 폴딩 바구니 ...
262376,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T21:24:51+09:00,"{""products"":[{""name"":""프리미엄 로즈힙 퓨레 엘라스틴 600mg 6...",{},NaN,NaN,NaN,NaN,"[{""name"":""프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘...",NaN,"프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘라스틴 60정 2병",31330.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff5e906-06ea-44ed-ae01-c4c3e6c4a821,Android,66482903-6256-4092-9d20-1e220804f289,"프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘라스틴 60정 2병"
497648,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T17:52:33+09:00,"{""products"":[{""name"":""여성 봄 정장 자켓"",""price"":6300...",{},NaN,NaN,NaN,NaN,"[{""name"":""여성 봄 정장 자켓"",""price"":63000}]",NaN,여성 봄 정장 자켓,63000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff8f4f4-8256-4df3-a923-fef6b12a090d,Android,047d6ed4-a627-4d73-bc59-1467313bfde3,여성 봄 정장 자켓
243776,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T21:37:36+09:00,"{""products"":[{""name"":""메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹...",{},NaN,NaN,NaN,NaN,"[{""name"":""메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정,...",NaN,"메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정, 다크브라운, 3개",29100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fffaeb83-177a-4747-bb1d-61b9e9377010,Android,ff8d11c2-4265-4f9c-9388-80790643f799,"메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정, 다크브라운, 3개"


### 상품 정보 조회 
단순 머지로 30건 미만 구매..? 미친;


In [111]:
cp.rename(columns={'date':'Event Date', 'subParam':'User ID', 'productName':'prev_prop'},inplace=True)
df['Event Date'] = pd.to_datetime(df['Event Date'])

/var/folders/7z/944bgjcs659fsp58nc954qgm0000gn/T/ipykernel_50424/2145641118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cp.rename(columns={'date':'Event Date', 'subParam':'User ID', 'productName':'prev_prop'},inplace=True)


In [56]:

cp_sp = cp.query('date >= "2024-04-12"')
tst = list(set(cp.query('date >= "2024-04-12"').subParam.tolist()))
df_sp = df.query('`Event Date` >="2024-04-12" & `User ID`in @tst')
df_sp['Event Date'] = pd.to_datetime(df_sp['Event Date'])
cp_sp.rename(columns={'date':'Event Date', 'subParam':'User ID', 'productName':'prev_prop'},inplace=True)


/var/folders/7z/944bgjcs659fsp58nc954qgm0000gn/T/ipykernel_50424/4028975924.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sp['Event Date'] = pd.to_datetime(df_sp['Event Date'])
/var/folders/7z/944bgjcs659fsp58nc954qgm0000gn/T/ipykernel_50424/4028975924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cp.rename(columns={'date':'Event Date', 'subParam':'User ID', 'productName':'prev_prop'},inplace=True)


In [223]:
cp.head()

,Event Date,trackingCode,subId,User ID,addtag,ctag,orderId,productId,prev_prop,quantity,gmv,commissionRate,commission
10842,2024-04-12,AF3155932,homecoin,bdaceb8d-3ea3-45ca-afde-ca09661083a7,460,7974963103,8074944516100772,7670376838,양반 단호박죽,6,10060.0,5.25,529.0
10843,2024-04-12,AF3155932,homecoin,0c089cc6-ff4e-4637-8c68-1549d696bb3f,460,7041059088,8814944016749336,7692530333,[식약처기능인정] 뱃살타파 2주완성 마시는 레몬밤 오비엑스 워터믹스 클래식,1,23500.0,5.25,1234.0
10844,2024-04-12,AF3155932,homecoin,614de0bf-119e-4df8-85b1-017420830eed,460,6489890342,8364941016020644,7292466704,엘라스틴 데일리 리페어링 퍼퓸 샴푸+린스 1100ml 대용량,1,16430.0,5.25,863.0
10845,2024-04-12,AF3155932,homecoin,e2b9c525-2b2d-4154-ae4a-772d08a5b414,460,6990601217,8004943116446031,6320364957,워너프 탱크 베어 링 휴대폰 케이스,1,8910.0,5.25,468.0
10846,2024-04-12,AF3155932,homecoin,b077a3e4-d241-4a2d-9e41-1c120bbae424,460,6957293598,8604948416503210,140401459,미샤 롱웨어 젤 펜슬 라이너 0.14g,1,3940.0,5.25,207.0


In [231]:
# 클릭과 구매 데이터 조인
converted_clicks = pd.merge(cp, df_sp, on=['Event Date', 'User ID', 'prev_prop'], how='inner')

# 결과 출력
print(converted_clicks)
converted_clicks

   Event Date trackingCode     subId                               User ID  \
0  2024-04-12    AF3155932  homecoin  0530aae5-57a1-4744-8803-eacbf6122035   
1  2024-04-13    AF3155932  homecoin  12027603-ede7-4b07-b010-8202b0a95fca   
2  2024-04-13    AF3155932  homecoin  b31fbf93-a31e-4a61-b90e-3015582b23bd   
3  2024-04-13    AF3155932  homecoin  7b78b443-d6c6-4401-b3c9-424e9c68879b   
4  2024-04-13    AF3155932  homecoin  ea597022-ccc1-4a94-a5f4-57e9deb56e9f   
5  2024-04-13    AF3155932  homecoin  53e0e30f-cb74-486f-8783-97d41e8858bb   
6  2024-04-13    AF3155932  homecoin  9a173147-b346-4edb-9020-b1755f2041ce   
7  2024-04-13    AF3155932  homecoin  4778c416-3cdb-48de-a819-6227a349ddc3   
8  2024-04-13    AF3155932  homecoin  7358b4ed-b70d-4791-851f-69ae0cf86653   
9  2024-04-13    AF3155932  homecoin  4778c416-3cdb-48de-a819-6227a349ddc3   
10 2024-04-14    AF3155932  homecoin  dba9a793-a9a2-424c-ac4f-c548e9df0193   
11 2024-04-15    AF3155932  homecoin  822e0831-e7d6-4f42-9bdf-09

,Event Date,trackingCode,subId,User ID,addtag,ctag,orderId,productId,prev_prop,quantity,gmv,commissionRate,commission,Event Category,Event Action,Event Label,Event Value,Event Datetime,Semantic Event Properties,Custom Event Properties,Total Revenue,Total Revenue (Original),Currency,Currency (Original),Product List,Product ID,Product Name,Product Price,Product Quantity,Product Currency,Product Position,Product Brand ID,Product Brand Name,Product Category ID,Product Category Name,Airbridge Device ID,Platform
0,2024-04-12,AF3155932,homecoin,0530aae5-57a1-4744-8803-eacbf6122035,460,6357887760,8964949516670924,6357887760,에킨 캘빈 조거 트레이닝팬츠 M~3XL 면 빅사이즈 운동복 츄리닝 사계절용 남녀공용,1,29800.0,5.25,1565.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-12T22:21:00+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d5f571cb-7da6-4f5c-9f62-0fa657bb9f5b,Android
1,2024-04-13,AF3155932,homecoin,12027603-ede7-4b07-b010-8202b0a95fca,460,7928006747,8234941716160407,7928006747,[스타일필드] 4세트 봄 여름 시즌 쿨 고스판 냉감 긴팔 라운드 티셔츠 일상복 작업...,1,28900.0,5.25,1518.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-13T18:24:37+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2097ee9e-2541-4323-a284-83dfe9f4d926,Android
2,2024-04-13,AF3155932,homecoin,b31fbf93-a31e-4a61-b90e-3015582b23bd,400,7708595421,8094949716356199,7708595421,더담은 2023년 햅쌀 든든한 밥상 전라미,1,22900.0,5.25,1203.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-13T22:58:52+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37db8087-b747-49b6-b3f0-83021246bf02,Android
3,2024-04-13,AF3155932,homecoin,7b78b443-d6c6-4401-b3c9-424e9c68879b,460,7759718455,8944940516219319,7420163115,알루미늄 탑박스 경첩세트 2set 교체용,1,8500.0,5.25,447.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-13T21:27:48+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ae7db2c8-0ff0-4eba-8728-7341d99771a8,Android
4,2024-04-13,AF3155932,homecoin,ea597022-ccc1-4a94-a5f4-57e9deb56e9f,460,7758218137,8214940516040544,7758218137,[clartte] 초경량 데일리 발편한 여성 운동화,1,16800.0,5.25,882.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-13T22:46:17+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79a306f4-04ff-498d-997b-5c792b2183c1,Android
5,2024-04-13,AF3155932,homecoin,53e0e30f-cb74-486f-8783-97d41e8858bb,460,7093624361,8674940810860335,7093624361,홍시루 수제 녹두 메밀 찰떡 1kg 달지않은 건강 떡 아침 식사 대용 간식,2,36000.0,5.25,1890.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-13T23:20:58+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c2da56a3-5caf-4f0f-bf0f-45e2c9af156f,Android
6,2024-04-13,AF3155932,homecoin,9a173147-b346-4edb-9020-b1755f2041ce,460,7170679762,8084945016446742,7170679762,에이블팩토리 아동용 논슬립 유치원 버스와 짱구 벨크로 실내화,1,9830.0,5.25,517.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-13T10:04:25+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c2cb78d5-701c-40a9-a6d8-521ba176b876,Android
7,2024-04-13,AF3155932,homecoin,4778c416-3cdb-48de-a819-6227a349ddc3,460,7815636404,8924943616876858,7815636404,에바주얼리 14k 골드필드 통통 골든 금 볼 팔찌 3mm 4mm,1,64000.0,5.25,3360.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-13T18:09:11+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1c25f230-ca02-4334-9f61-16e90ddce5be,Android
8,2024-04-13,AF3155932,homecoin,7358b4ed-b70d-4791-851f-69ae0cf86653,460,7854083799,8654947516509727,7854083799,ANBER 여성용 가죽 (굽 9cm) 키높이 스니커즈,1,39800.0,5.25,2090.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-13T09:43:14+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8a82a74d-0d03-4a78-95df-9644677468a6,Android
9,2024-04-13,AF3155932,homecoin,4778c416-3cdb-48de-a819-6227a349ddc3,460,7815636404,8924943916186642,7815636404,에바주얼리 14k 골드필드 통통 골든 금 볼 팔찌 3mm 4mm,1,64000.0,5.25,3360.0,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-13T18:09:11+09:00,"{""products"":[]}",{},NaN,NaN

유사도

In [ ]:

import pandas as pd

# 클릭 데이터와 구매 데이터를 로드 (여기서는 예제 데이터 사용)
clicks = pd.DataFrame({
    'date': ['2024-01-01', '2024-01-01', '2024-01-02'],
    'userid': [1, 2, 1],
    'productname': ['Product A', 'Product B', 'Product C']
})

purchases = pd.DataFrame({
    'date': ['2024-01-01', '2024-01-01'],
    'userid': [1, 1],
    'productname': ['Product AA', 'Product BB']
})

# 유사도 기준으로 클릭과 구매 매칭
def match_products(click_row, purchases):
    # 가장 유사한 제품 이름 찾기
    filtered_purchases = purchases[(purchases['Event Date'])]

# 클릭 데이터에 매칭된 구매 제품 이름 추가
clicks['matched_purchase'] = clicks.apply(match_products, purchases=purchases, axis=1)

# 결과 확인
print(clicks)


In [151]:
df = df.query("`Event Date` > '2024-04-12' ")
cp = cp.query("`Event Date` > '2024-04-12' ")

In [108]:
# tmp_df = df[df['Event Date']>="2024-04-20"]
# tmp_df['prev_prop'] = tmp_df['Product Name']
# tmp_df
df = df[df['Event Date']<"2024-04-20"]
df = pd.concat([df,tmp_df],axis=0)

In [109]:
df.tail()

,Event Category,Event Action,Event Label,Event Value,Event Date,Event Datetime,Semantic Event Properties,Custom Event Properties,Total Revenue,Total Revenue (Original),Currency,Currency (Original),Product List,Product ID,Product Name,Product Price,Product Quantity,Product Currency,Product Position,Product Brand ID,Product Brand Name,Product Category ID,Product Category Name,Airbridge Device ID,Platform,User ID,prev_prop
676390,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T00:00:06+09:00,"{""products"":[{""name"":""미스터장어 국산 자포니카 민물장어 1kg 손...",{},NaN,NaN,NaN,NaN,"[{""name"":""미스터장어 국산 자포니카 민물장어 1kg 손질 후 750g내외, ...",NaN,"미스터장어 국산 자포니카 민물장어 1kg 손질 후 750g내외, 1kg (손질후 7...",85800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,661468a3-16a4-416b-92b3-28613da2be1e,Android,be0dcfb0-fd2c-4ad4-a0db-ccca20f96cbb,"미스터장어 국산 자포니카 민물장어 1kg 손질 후 750g내외, 1kg (손질후 7..."
676402,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T00:00:05+09:00,"{""products"":[{""name"":""[행복미트] 호주산 프리미엄 소 스지 알스지...",{},NaN,NaN,NaN,NaN,"[{""name"":""[행복미트] 호주산 프리미엄 소 스지 알스지 1kg, 수육용 탕용...",NaN,"[행복미트] 호주산 프리미엄 소 스지 알스지 1kg, 수육용 탕용 진공포장 [당일발...",25640.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9edd71da-70c4-4a6e-8237-c054b296561e,Android,2b0a7c2a-4f5f-4970-8279-b4cf46624596,"[행복미트] 호주산 프리미엄 소 스지 알스지 1kg, 수육용 탕용 진공포장 [당일발..."
676403,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T00:00:05+09:00,"{""products"":[{""name"":""화이트스톤 돔 맥세이프 삼각대 블루투스 셀카...",{},NaN,NaN,NaN,NaN,"[{""name"":""화이트스톤 돔 맥세이프 삼각대 블루투스 셀카봉 4in1, 단일상품...",NaN,"화이트스톤 돔 맥세이프 삼각대 블루투스 셀카봉 4in1, 단일상품",23000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5d852c26-e84f-4e05-85d8-e6a9a71e1ef2,Android,f211a4ef-7011-48fa-8489-1fd70ae61e39,"화이트스톤 돔 맥세이프 삼각대 블루투스 셀카봉 4in1, 단일상품"
676494,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T00:00:00+09:00,"{""products"":[{""name"":""메디레머디 사철쑥53 주름개선 모이스처 토너...",{},NaN,NaN,NaN,NaN,"[{""name"":""메디레머디 사철쑥53 주름개선 모이스처 토너 500ml, 1개"",...",NaN,"메디레머디 사철쑥53 주름개선 모이스처 토너 500ml, 1개",16470.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1CD6AC07-B2D6-4CB9-907C-31FED9CEB230,iOS,34466e2d-12b4-49de-b64b-503ff1d8a968,"메디레머디 사철쑥53 주름개선 모이스처 토너 500ml, 1개"
676496,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T00:00:00+09:00,"{""products"":[{""name"":""[수이골프 I형] 스포츠 마스크 골프 등산 ...",{},NaN,NaN,NaN,NaN,"[{""name"":""[수이골프 I형] 스포츠 마스크 골프 등산 조깅 바이크, I-3 ...",NaN,"[수이골프 I형] 스포츠 마스크 골프 등산 조깅 바이크, I-3 체인 블랙, M, M",23400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,528A3C64-9E1D-4CE6-ADF2-E0BDBA27155F,iOS,b080f311-4ab0-4a5a-93f1-303e0572387b,"[수이골프 I형] 스포츠 마스크 골프 등산 조깅 바이크, I-3 체인 블랙, M, M"


In [112]:
df_sp = df[['Event Date', 'User ID', 'prev_prop']]
df_sp = df_sp.query('prev_prop.notna()')
cp_sp = cp[['Event Date', 'User ID', 'prev_prop']]


In [119]:
df_sp.iloc[[1]]['Event Date']

663043   2024-04-08
Name: Event Date, dtype: datetime64[ns]

구매 상품과 유사도 매칭. 

In [91]:
### 당일만 체크
def match_products(row, cp):
    # 해당 날짜와 사용자 ID에 대한 구매 데이터 필터링
    filtered_purchase = cp[(cp['Event Date'] == row['Event Date']) & (cp['User ID'] == row['User ID'])]
    
    # 필터링된 구매 데이터가 비어있지 않은 경우
    if not filtered_purchase.empty and pd.notna(row['prev_prop']):
        # 가장 유사한 제품 이름 찾기
        valid_purchase_names = filtered_purchase['prev_prop'].dropna().tolist()
        if valid_purchase_names:
            match = process.extractOne(row['prev_prop'], valid_purchase_names, score_cutoff=80)
            if match:
                return match[0], match[1]
    return None, None  # 항상 두 개의 값을 반환하도록 수정

# 클릭 데이터에 매칭된 구매 제품 이름과 점수 추가
df_sp[['matched_purchase', 'match_score']] = df_sp.apply(
    lambda row: match_products(row, cp_sp), axis=1, result_type='expand'
)


In [120]:
### d+1일까지 체크
import pandas as pd
from fuzzywuzzy import process

def match_products(row, cp):
    # 클릭 날짜와 그 다음 날짜를 포함하여 데이터 필터링
    next_day = row['Event Date'] + pd.Timedelta(days=1)  # 클릭 다음 날
    filtered_purchase = cp[(cp['Event Date'].isin([row['Event Date'], next_day])) & (cp['User ID'] == row['User ID'])]
    
    # 필터링된 구매 데이터가 비어있지 않은 경우
    if not filtered_purchase.empty and pd.notna(row['prev_prop']):
        # 가장 유사한 제품 이름 찾기
        valid_purchase_names = filtered_purchase['prev_prop'].dropna().tolist()
        if valid_purchase_names:
            match = process.extractOne(row['prev_prop'], valid_purchase_names, score_cutoff=60)
            if match:
                return match[0], match[1]
    return None, None  # 항상 두 개의 값을 반환하도록 수정

# 클릭 데이터에 매칭된 구매 제품 이름과 점수 추가
df_sp[['matched_purchase', 'match_score']] = df_sp.apply(
    lambda row: match_products(row, cp_sp), axis=1, result_type='expand'
)


In [175]:
df_sp[df_sp['matched_purchase'].notna()]

,Event Date,User ID,prev_prop,matched_purchase,match_score
443631,2024-04-12,659eb12f-ffa8-4c8e-b7a8-13db74856700,"미스터장어 국산 자포니카 민물장어 1kg 손질 후 750g내외, 1kg (손질후 7...",이더스 열풍으로 구운 국산 볶은 서리태,86.0
658257,2024-04-12,c0ebab3a-2c88-44af-a11e-acc98b3350b4,"바세린 립 테라피 코코아 립밤 7g, 코코아 버터 향, 7g, 1개",바세린 프로텍팅 젤리 코코아 버터,86.0
192430,2024-04-12,ac2fd928-e427-4104-b8e6-2127c31dd72b,"[로켓프레시] 미식연구소 갓튀겨나온 곱빼기 통등심 젓가락 돈까스 (냉동), 500g...",풀무원 쌀 물만두 (냉동),86.0
681333,2024-04-12,fb4e735f-4cca-40ec-8134-18e5883553a7,"라입 R3 아르간 헤어오일 베이비파우더, 100ml, 3개",라입 R3 아르간오일 베이비파우더 헤어에센스,68.0
469847,2024-04-12,e8478a18-2abf-4761-b902-33353c4cca42,"[로켓프레시] 허닭 스팀 닭가슴살 슬라이스 케이준맛 (냉동), 100g, 20팩",하림 IFF 한입쏙 닭안심 블랙페퍼 (냉동),86.0
670083,2024-04-12,744fd655-98a6-4022-aff7-c5059219fc7b,[소중한가족] 조용하고 고정이편한 밴드형 침대 매트리스 방수 커버 패드 SS Q K,프로텍트어베드 방수 언더패드,86.0
553784,2024-04-12,3dbae5f5-793d-4e7c-b107-c9d1b2f82594,엘엑스티 남성용 구김없는 반팔 와이셔츠,아르메데스 남성용 기능성 슈퍼 드라이 반팔 티셔츠 AR-192,86.0
429298,2024-04-12,5de00b00-d396-4e50-91fa-46b94c96af51,농심 짜파게티,농심 신라면 블랙 사발 컵라면 101 g,86.0
69125,2024-04-12,93fd27e5-526e-4127-b09d-5a0af79df521,"[로켓프레시] 미식연구소 갓튀겨나온 곱빼기 통등심 젓가락 돈까스 (냉동), 500g...",곰곰 고기산적 (냉동),86.0
411040,2024-04-12,8152cf88-28a6-4fa8-8481-de0466a7eb8d,"살림생각 거꾸로 물빠짐 칫솔 양치컵 물컵 거치대, 2개입, 투명",편한오늘 거꾸로 양치컵 칫솔꽂이 스텐드,86.0


In [167]:
df_sp[df_sp['matched_purchase'].notna()].groupby('Event Date')[['matched_purchase']].count()

,matched_purchase
Event Date,
2024-04-12,70
2024-04-13,100
2024-04-14,73
2024-04-15,80
2024-04-16,104
2024-04-17,102
2024-04-18,75
2024-04-19,41
2024-04-20,44


In [168]:
cp.columns

Index(['Event Date', 'trackingCode', 'subId', 'User ID', 'addtag', 'ctag',
       'orderId', 'productId', 'prev_prop', 'quantity', 'gmv',
       'commissionRate', 'commission'],
      dtype='object')

In [174]:
cp_purchase_cnt = cp.groupby('Event Date')[['orderId']].count()
df_purchase_cnt = df_sp[df_sp['matched_purchase'].notna()].groupby('Event Date')[['matched_purchase']].count()

df_purchase_cnt = df_purchase_cnt.merge(cp_purchase_cnt,left_index=True,right_index=True)
df_purchase_cnt['ratio'] = round(df_purchase_cnt['matched_purchase'] / df_purchase_cnt['orderId'],2)
print(df_purchase_cnt.ratio.mean())
df_purchase_cnt

0.027777777777777776


,matched_purchase,orderId,ratio
Event Date,,,
2024-04-13,100,2863,0.03
2024-04-14,73,2776,0.03
2024-04-15,80,2909,0.03
2024-04-16,104,3127,0.03
2024-04-17,102,3160,0.03
2024-04-18,75,2405,0.03
2024-04-19,41,1820,0.02
2024-04-20,44,1599,0.03
2024-04-21,28,1358,0.02


In [97]:
df_sp[df_sp['Event Date']=="2024-04-21"]

,Event Date,User ID,prev_prop,matched_purchase,match_score


유저아이디 보간 처리해도 20%가량 빔.

In [ ]:
# Airbridge Device ID 별로 첫 번째 유효한 User ID 찾기
valid_user_id = df.dropna(subset=['User ID']).drop_duplicates('Airbridge Device ID').set_index('Airbridge Device ID')['User ID']

# 원본 데이터프레임에 매핑된 User ID를 병합하기
df = df.merge(valid_user_id, on='Airbridge Device ID', how='left', suffixes=('', '_filled'))

# 비어 있는 User ID 채우기
df['User ID'] = df['User ID'].fillna(df['User ID_filled'])

# 불필요한 컬럼 제거
df.drop(columns=['User ID_filled'], inplace=True)

# 결과 출력
print(df)

In [138]:
df[df['User ID'].isna()]['Airbridge Device ID'].nunique() / df['Airbridge Device ID'].nunique()

0.1929997653947635